# Notebook 2: GCN, GraphSAGE, and GAT

This notebook covers three foundational **Graph Neural Network** architectures:

| Model | Paper | Key Idea |
|-------|-------|----------|
| **GCN** | Kipf & Welling (2017) | Spectral convolution with symmetric normalisation |
| **GraphSAGE** | Hamilton et al. (2017) | Inductive learning via sampled neighbourhood aggregation |
| **GAT** | Veličković et al. (2018) | Attention-weighted aggregation |

**Contents**
1. [Graph Convolutional Network (GCN)](#1-graph-convolutional-network-gcn)
2. [GraphSAGE](#2-graphsage)
3. [Graph Attention Network (GAT)](#3-graph-attention-network-gat)
4. [Comparison on Cora](#4-comparison-on-cora)
5. [Exercises](#5-exercises)

In [ ]:
# Uncomment to install
# !pip install torch torch_geometric

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

print(f'PyTorch version: {torch.__version__}')

try:
    import torch_geometric
    print(f'PyG version: {torch_geometric.__version__}')
    pyg_available = True
except ImportError:
    print('PyTorch Geometric not installed — some cells will be skipped')
    pyg_available = False

In [ ]:
if pyg_available:
    from torch_geometric.datasets import Planetoid
    from torch_geometric.nn import GCNConv, SAGEConv, GATConv
    import torch_geometric.transforms as T

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    dataset = Planetoid(root='/tmp/Cora', name='Cora',
                        transform=T.NormalizeFeatures())
    data = dataset[0].to(device)

    print(data)
    print(f'Classes: {dataset.num_classes}')
    print(f'Node features: {dataset.num_features}')

---
## 1. Graph Convolutional Network (GCN)

### 1.1 Theoretical Background

#### 1.1.1 Spectral Graph Theory Motivation

Classical CNNs apply learnable filters in the spatial domain on grid-structured data (images). To generalise convolution to graphs, one approach uses the **graph Laplacian**:

$$
\mathbf{L} = \mathbf{D} - \mathbf{A}
$$

where $\mathbf{D}$ is the diagonal degree matrix. The normalised Laplacian $\tilde{\mathbf{L}} = \mathbf{D}^{-1/2}\mathbf{L}\mathbf{D}^{-1/2}$ is symmetric positive semi-definite with eigendecomposition $\tilde{\mathbf{L}} = \mathbf{U}\boldsymbol{\Lambda}\mathbf{U}^\top$.

A spectral graph convolution is defined as:
$$\mathbf{x} \star g = \mathbf{U}\,g(\boldsymbol{\Lambda})\,\mathbf{U}^\top \mathbf{x}$$

This is expensive ($O(N^2)$ for eigen-decomposition). Kipf & Welling simplify by using a first-order approximation and adding self-loops.

#### 1.1.2 GCN Propagation Rule

The final layer-wise propagation rule is:

$$
\boxed{\mathbf{H}^{(l+1)} = \sigma\!\left(
    \hat{\mathbf{D}}^{-1/2}\hat{\mathbf{A}}\hat{\mathbf{D}}^{-1/2}\mathbf{H}^{(l)}\mathbf{W}^{(l)}
\right)}
$$

| Symbol | Meaning |
|--------|---------|
| $\hat{\mathbf{A}} = \mathbf{A} + \mathbf{I}$ | Adjacency matrix with added self-loops |
| $\hat{\mathbf{D}}_{ii} = \sum_j \hat{\mathbf{A}}_{ij}$ | Degree matrix of $\hat{\mathbf{A}}$ |
| $\mathbf{W}^{(l)}$ | Learnable weight matrix |
| $\sigma$ | Activation function (e.g., ReLU) |

**Message-passing interpretation:**  For each node $v$:
$$
h_v^{(l+1)} = \sigma\!\left(
    \sum_{u \in \mathcal{N}(v) \cup \{v\}} \frac{1}{\sqrt{\hat{d}_v \hat{d}_u}} \mathbf{W}^{(l)} h_u^{(l)}
\right)
$$

Each node aggregates its neighbours' features, normalised by degree.

#### 1.1.3 Limitations
- **Transductive**: the model must see all nodes during training (fixed graph)
- **Depth**: stacking many layers causes **over-smoothing** (all node representations converge)
- **Symmetric normalisation**: treats all neighbours equally

### 1.2 GCN Implementation from Scratch

In [ ]:
import scipy.sparse as sp
import numpy as np

def gcn_norm(edge_index, num_nodes):
    """Compute D^{-1/2} A_hat D^{-1/2} as sparse edge weights."""
    # Build adjacency with self-loops
    row = torch.cat([edge_index[0], torch.arange(num_nodes)])
    col = torch.cat([edge_index[1], torch.arange(num_nodes)])

    # Degree
    deg = torch.zeros(num_nodes)
    deg.scatter_add_(0, row, torch.ones(row.size(0)))
    deg_inv_sqrt = deg.pow(-0.5)

    norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
    new_edge_index = torch.stack([row, col], dim=0)
    return new_edge_index, norm


class GCNLayerFromScratch(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.W = nn.Linear(in_ch, out_ch, bias=False)

    def forward(self, x, edge_index, norm):
        # 1. Linear transform
        x = self.W(x)                          # [N, out_ch]
        # 2. Aggregate (manual scatter)
        row, col = edge_index
        agg = torch.zeros_like(x)
        agg.scatter_add_(0, col.unsqueeze(-1).expand_as(x[row]),
                         norm.unsqueeze(-1) * x[row])
        return agg


print('GCNLayerFromScratch defined successfully')

### 1.3 GCN with PyG's `GCNConv`

In [ ]:
if pyg_available:
    class GCN(nn.Module):
        def __init__(self, in_ch, hidden_ch, out_ch, dropout=0.5):
            super().__init__()
            self.conv1 = GCNConv(in_ch, hidden_ch)
            self.conv2 = GCNConv(hidden_ch, out_ch)
            self.dropout = dropout

        def forward(self, x, edge_index):
            x = F.relu(self.conv1(x, edge_index))
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)

    print('GCN model defined')

---
## 2. GraphSAGE

### 2.1 Theoretical Background

**GraphSAGE** (Hamilton et al., 2017) — *SAmple and aggreGatE* — was designed for **inductive** learning: it can generalise to unseen nodes at test time without retraining.

#### 2.1.1 Key Idea: Neighbourhood Sampling

Instead of using the full neighbourhood (expensive for hub nodes), GraphSAGE samples a **fixed-size** set of neighbours $\mathcal{S}(v) \subseteq \mathcal{N}(v)$.

#### 2.1.2 Propagation Rule

$$
\mathbf{h}_{\mathcal{N}(v)}^{(k)} = \text{AGGREGATE}^{(k)}\!\left(
    \left\{\mathbf{h}_u^{(k-1)} : u \in \mathcal{S}(v)\right\}
\right)
$$

$$
\mathbf{h}_v^{(k)} = \sigma\!\left(
    \mathbf{W}^{(k)} \cdot \text{CONCAT}\!\left(\mathbf{h}_v^{(k-1)},\, \mathbf{h}_{\mathcal{N}(v)}^{(k)}\right)
\right)
$$

$$
\mathbf{h}_v^{(k)} \leftarrow \frac{\mathbf{h}_v^{(k)}}{\|\mathbf{h}_v^{(k)}\|_2}
$$

#### 2.1.3 Aggregators

| Aggregator | Formula |
|------------|----------|
| **Mean** | $\frac{1}{|\mathcal{S}(v)|}\sum_{u}\mathbf{h}_u$ |
| **Max-pool** | $\max(\{\sigma(\mathbf{W}_{\text{pool}}\mathbf{h}_u + \mathbf{b}) : u \in \mathcal{S}(v)\})$ |
| **LSTM** | Apply LSTM on a random permutation of $\mathcal{S}(v)$ |

#### 2.1.4 Inductive Setting

The key advantage over GCN: the weight matrices $\mathbf{W}^{(k)}$ are shared across all nodes. Given a new node, we only need its features and a sample of its neighbours — **no retraining required**.

#### 2.1.5 Unsupervised Training via Graph-based Loss

For unsupervised learning, GraphSAGE uses a **binary cross-entropy** loss:
$$
J = -\log\sigma(\mathbf{z}_u^\top \mathbf{z}_v) - Q \cdot \mathbb{E}_{v_n \sim P_n}[\log\sigma(-\mathbf{z}_u^\top \mathbf{z}_{v_n})]
$$
where $v$ is a nearby node (short random walk) and $v_n$ is a noise node.

### 2.2 GraphSAGE with PyG

In [ ]:
if pyg_available:
    class GraphSAGE(nn.Module):
        def __init__(self, in_ch, hidden_ch, out_ch, dropout=0.5):
            super().__init__()
            # aggr='mean' by default; also try 'max' or 'lstm'
            self.conv1 = SAGEConv(in_ch, hidden_ch, aggr='mean')
            self.conv2 = SAGEConv(hidden_ch, out_ch, aggr='mean')
            self.dropout = dropout

        def forward(self, x, edge_index):
            x = F.relu(self.conv1(x, edge_index))
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)

    print('GraphSAGE model defined')

### 2.3 Inductive Learning with Neighbourhood Sampling

In [ ]:
if pyg_available:
    try:
        from torch_geometric.loader import NeighborLoader

        # NeighborLoader samples a fixed number of neighbours per hop
        train_loader = NeighborLoader(
            data,
            num_neighbors=[10, 5],   # 10 neighbours in hop-1, 5 in hop-2
            batch_size=256,
            input_nodes=data.train_mask,
        )

        # Peek at a mini-batch
        for mini_batch in train_loader:
            print('Mini-batch:', mini_batch)
            break
    except Exception as e:
        print(f'NeighborLoader demo skipped: {e}')

---
## 3. Graph Attention Network (GAT)

### 3.1 Theoretical Background

**GAT** (Veličković et al., 2018) introduces **attention coefficients** so that different neighbours contribute differently to each node's new representation.

#### 3.1.1 Attention Mechanism

Given node features $\mathbf{h}_i, \mathbf{h}_j \in \mathbb{R}^F$:

**Step 1 – Linear projection:**
$$\mathbf{z}_i = \mathbf{W}\mathbf{h}_i, \quad \mathbf{z}_j = \mathbf{W}\mathbf{h}_j$$

**Step 2 – Attention coefficient:**
$$e_{ij} = \text{LeakyReLU}\!\left(\mathbf{a}^\top [\mathbf{z}_i \| \mathbf{z}_j]\right)$$

**Step 3 – Normalise with Softmax (over neighbours):**
$$\alpha_{ij} = \text{softmax}_j(e_{ij}) = \frac{\exp(e_{ij})}{\sum_{k \in \mathcal{N}(i)} \exp(e_{ik})}$$

**Step 4 – Weighted aggregation:**
$$\mathbf{h}_i^{\prime} = \sigma\!\left(\sum_{j \in \mathcal{N}(i)} \alpha_{ij} \mathbf{z}_j\right)$$

#### 3.1.2 Multi-Head Attention

To stabilise training, GAT uses $K$ independent attention heads:

$$\mathbf{h}_i^{\prime} = \big\|_{k=1}^K \sigma\!\left(\sum_{j \in \mathcal{N}(i)} \alpha_{ij}^{(k)} \mathbf{W}^{(k)}\mathbf{h}_j\right)$$

For the final layer, averaging is used instead of concatenation:
$$\mathbf{h}_i^{\prime} = \sigma\!\left(\frac{1}{K}\sum_{k=1}^K \sum_{j \in \mathcal{N}(i)} \alpha_{ij}^{(k)} \mathbf{W}^{(k)}\mathbf{h}_j\right)$$

#### 3.1.3 GATv2

Brody et al. (2022) proposed **GATv2**, which fixes an expressiveness issue in the original GAT. The attention score is computed as:
$$e_{ij} = \mathbf{a}^\top \text{LeakyReLU}\!\left(\mathbf{W}[\mathbf{h}_i \| \mathbf{h}_j]\right)$$

This is a *dynamic* attention mechanism (the linear projection happens *after* concatenation).

### 3.2 GAT from Scratch (Illustrative)

In [ ]:
class GATLayerFromScratch(nn.Module):
    """Single-head GAT layer (illustrative, not optimised)."""

    def __init__(self, in_features, out_features, dropout=0.6, alpha=0.2):
        super().__init__()
        self.W = nn.Linear(in_features, out_features, bias=False)
        # Attention vector: a in R^{2*out_features}
        self.a = nn.Parameter(torch.empty(2 * out_features))
        nn.init.xavier_uniform_(self.a.unsqueeze(0))
        self.leaky_relu = nn.LeakyReLU(alpha)
        self.dropout = dropout

    def forward(self, x, edge_index):
        N = x.size(0)
        z = self.W(x)                          # [N, out_features]

        row, col = edge_index                  # source, target
        # Concatenate [z_i || z_j] for each edge
        z_cat = torch.cat([z[row], z[col]], dim=-1)  # [E, 2*out]
        e = self.leaky_relu((z_cat * self.a).sum(-1)) # [E]

        # Softmax over incoming edges for each target node
        # Use scatter_softmax from PyG or manual implementation
        # (manual for illustration)
        e_max = torch.full((N,), float('-inf'))
        e_max.scatter_reduce_(0, col, e, reduce='amax', include_self=True)
        e_exp = (e - e_max[col]).exp()
        e_sum = torch.zeros(N).scatter_add_(0, col, e_exp)
        alpha = e_exp / (e_sum[col] + 1e-16)

        # Weighted aggregation
        out = torch.zeros_like(z)
        out.scatter_add_(0, col.unsqueeze(-1).expand_as(z[row]),
                         alpha.unsqueeze(-1) * z[row])
        return F.elu(out)


print('GATLayerFromScratch defined')

### 3.3 GAT with PyG's `GATConv`

In [ ]:
if pyg_available:
    class GAT(nn.Module):
        def __init__(self, in_ch, hidden_ch, out_ch, heads=8, dropout=0.6):
            super().__init__()
            # First layer: 8 heads, concat=True => hidden_ch * heads output
            self.conv1 = GATConv(in_ch, hidden_ch, heads=heads,
                                 dropout=dropout, concat=True)
            # Final layer: 1 head, concat=False => average
            self.conv2 = GATConv(hidden_ch * heads, out_ch, heads=1,
                                 dropout=dropout, concat=False)
            self.dropout = dropout

        def forward(self, x, edge_index):
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = F.elu(self.conv1(x, edge_index))
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)

    print('GAT model defined')

### 3.4 Visualising Attention Weights

In [ ]:
if pyg_available:
    # Inspect attention weights from a trained GAT
    # GATConv returns (out, (edge_index, alpha)) when return_attention_weights=True
    gat_layer = GATConv(dataset.num_features, 8, heads=1,
                        concat=False, return_attention_weights=True)
    gat_layer = gat_layer.to(device)

    gat_layer.eval()
    with torch.no_grad():
        out, (att_edge_index, att_weights) = gat_layer(data.x, data.edge_index,
                                                        return_attention_weights=True)

    print('Attention weights shape:', att_weights.shape)  # [num_edges, 1]
    print('Min attention:', att_weights.min().item())
    print('Max attention:', att_weights.max().item())

    plt.figure(figsize=(7, 4))
    plt.hist(att_weights.cpu().squeeze().numpy(), bins=50)
    plt.xlabel('Attention weight $\\alpha_{ij}$')
    plt.ylabel('Count')
    plt.title('Distribution of GAT Attention Weights on Cora')
    plt.show()

---
## 4. Comparison on Cora

### 4.1 Training Utility

In [ ]:
if pyg_available:
    def train_model(model, data, epochs=200, lr=0.01, wd=5e-4):
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        train_losses, val_accs = [], []

        for epoch in range(1, epochs + 1):
            model.train()
            optimizer.zero_grad()
            out = model(data.x, data.edge_index)
            loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

            model.eval()
            with torch.no_grad():
                pred = out.argmax(dim=1)
                val_acc = (pred[data.val_mask] == data.y[data.val_mask]).float().mean()
            val_accs.append(val_acc.item())

        return train_losses, val_accs

    @torch.no_grad()
    def evaluate(model, data):
        model.eval()
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        results = {}
        for split in ['train_mask', 'val_mask', 'test_mask']:
            mask = getattr(data, split)
            acc = (pred[mask] == data.y[mask]).float().mean().item()
            results[split.replace('_mask', '')] = acc
        return results

In [ ]:
if pyg_available:
    torch.manual_seed(42)

    models = {
        'GCN': GCN(dataset.num_features, 64, dataset.num_classes).to(device),
        'GraphSAGE': GraphSAGE(dataset.num_features, 64, dataset.num_classes).to(device),
        'GAT': GAT(dataset.num_features, 8, dataset.num_classes).to(device),
    }

    histories = {}
    for name, model in models.items():
        print(f'Training {name}...')
        losses, val_accs = train_model(model, data, epochs=200)
        histories[name] = (losses, val_accs)
        results = evaluate(model, data)
        print(f'  Train: {results["train"]:.4f} | Val: {results["val"]:.4f} | Test: {results["test"]:.4f}')

In [ ]:
if pyg_available and histories:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    for name, (losses, val_accs) in histories.items():
        axes[0].plot(losses, label=name)
        axes[1].plot(val_accs, label=name)

    axes[0].set_title('Training Loss'); axes[0].set_xlabel('Epoch'); axes[0].legend()
    axes[1].set_title('Validation Accuracy'); axes[1].set_xlabel('Epoch'); axes[1].legend()
    plt.tight_layout()
    plt.show()

### 4.2 Summary of Differences

| Property | GCN | GraphSAGE | GAT |
|----------|-----|-----------|-----|
| Neighbour weighting | Symmetric degree normalisation | Uniform (mean) or other | Learned attention |
| Self-loop handling | Explicit (add $\mathbf{I}$) | Concatenate self | Masked self-attention |
| Inductive capability | ✗ (transductive) | ✓ | ✓ (with caution) |
| Neighbour sampling | No | Yes | No |
| Computational cost | $O(|E| \cdot F)$ | $O(S \cdot F)$ | $O(|E| \cdot F)$ |
| # parameters | Low | Low | High (multi-head) |

---
## 5. Exercises

### Exercise 1 — Depth vs. Over-Smoothing in GCN

Train GCN models with 2, 4, 6, and 8 layers on Cora. For each depth:
1. Report test accuracy.
2. Compute the **mean cosine similarity** between all pairs of test node representations.
3. Plot accuracy and mean cosine similarity vs. depth.

Observe over-smoothing as depth increases.

In [ ]:
# Exercise 1 — your solution here
if pyg_available:
    class DeepGCN(nn.Module):
        def __init__(self, in_ch, hidden_ch, out_ch, num_layers):
            super().__init__()
            # TODO: build a GCN with num_layers layers
            ...

        def forward(self, x, edge_index):
            ...

### Exercise 2 — GraphSAGE Aggregators

Train three versions of GraphSAGE on Cora using different aggregators: `'mean'`, `'max'`, and `'lstm'`.
Compare their test accuracy and convergence speed.

In [ ]:
# Exercise 2 — your solution here
...

### Exercise 3 — Multi-Head Attention Ablation

Train GAT with `heads ∈ {1, 2, 4, 8}` on Cora. Plot test accuracy vs. number of heads.

In [ ]:
# Exercise 3 — your solution here
...

### Exercise 4 — Graph Classification with GIN

**Graph Isomorphism Network (GIN)** (Xu et al., 2019) is the most powerful GNN in the WL hierarchy:

$$h_v^{(k)} = \text{MLP}^{(k)}\!\left((1 + \epsilon^{(k)})\cdot h_v^{(k-1)} + \sum_{u \in \mathcal{N}(v)} h_u^{(k-1)}\right)$$

Using PyG's `GINConv` and global pooling:
1. Load the MUTAG dataset.
2. Build a 3-layer GIN classifier with `global_add_pool`.
3. Train for 100 epochs and compare with GCN-based classifier.

In [ ]:
# Exercise 4 — your solution here
if pyg_available:
    from torch_geometric.nn import GINConv, global_add_pool
    from torch_geometric.datasets import TUDataset
    from torch_geometric.loader import DataLoader as PyGDataLoader

    class GINClassifier(nn.Module):
        def __init__(self, in_ch, hidden_ch, out_ch):
            super().__init__()
            # TODO: 3 GINConv layers + MLP classifier
            ...

        def forward(self, x, edge_index, batch):
            # TODO
            ...

### Exercise 5 — Attention Map Visualisation

After training GAT on Cora:
1. Extract attention weights from both layers.
2. For a chosen node, visualise its ego-graph (1-hop neighbourhood) with edge thickness proportional to attention weight.

*Hint: use `networkx` and `matplotlib` with custom edge widths.*

In [ ]:
# Exercise 5 — your solution here
...

---
## Summary

| Model | Strengths | Weaknesses |
|-------|-----------|------------|
| **GCN** | Simple, fast, effective for dense/homophilic graphs | Transductive, over-smoothing, equal neighbour weighting |
| **GraphSAGE** | Inductive, scalable with sampling | Uniform aggregation (mean), needs careful sampling |
| **GAT** | Learned neighbour importance, interpretable | More parameters, sensitive to hyperparameters |

**Next notebook →** `03_knowledge_graph_embeddings.ipynb` — TransE, TransR, RotatE, ComplEx, LiteralE, and Heterogeneous Graphs.